In [76]:
import numpy as np
import pandas as pd
import gc
import seaborn as sns
import xgboost as xgb
import lightgbm as lgb
import math
import lightgbm as lgb
from collections import Counter  
import time
from scipy.stats import kurtosis,iqr
from scipy import ptp
from tqdm import tqdm
import datetime
from sklearn.metrics import accuracy_score, roc_auc_score,log_loss,f1_score
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
from sklearn.preprocessing import MinMaxScaler
import warnings
import os
from sklearn.utils import shuffle
from sklearn.externals import joblib
pd.options.display.max_columns = None
pd.options.display.max_rows = None
import warnings
warnings.filterwarnings('ignore')

In [77]:
### 北京市重点区域信息
cols = ['id', 'area_name', 'area_type', 'Center_x', 'Center_y', 'Grid_x', 'Grid_y', 'area']
area_passenger_info = pd.read_csv('/home/poac/AnomalyDetectionDataset/Block_predict/area_passenger_info.csv', names=cols)

In [78]:
area_passenger_info.iloc[0:5]

,id,area_name,area_type,Center_x,Center_y,Grid_x,Grid_y,area
0,1,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375
1,2,古塔公园,旅游景点;公园,116.535404,39.878676,116.535232,39.878140,575268.839844
2,3,中央戏剧学院,教育培训;高等院校,116.404667,39.935999,116.404148,39.936165,17024.636719
3,4,丰联广场,购物;购物中心,116.438556,39.923713,116.438117,39.924203,11695.019531
4,5,中华女子学院,教育培训;高等院校,116.435352,39.994935,116.434508,39.994636,83053.546875


In [79]:
### 重点区域人流量情况
cols = ['id', 'date/hour', 'index']
area_passenger_index = pd.read_csv('/home/poac/AnomalyDetectionDataset/Block_predict/area_passenger_index.csv', names=cols)
### 将时间戳拆成月，日和小时
### 去掉年份
area_passenger_index['month'] = area_passenger_index['date/hour'].apply(lambda x: int(''.join(str(x)[4:6])))
area_passenger_index['day'] = area_passenger_index['date/hour'].apply(lambda x: int(''.join(str(x)[6:8])))
area_passenger_index['hour'] = area_passenger_index['date/hour'].apply(lambda x: int(str(x)[8:]))

### 删除原时间戳
area_passenger_index = area_passenger_index.drop(['date/hour'], axis=1)

# area_passenger_index['date/hour'] = area_passenger_index['date/hour'].apply(lambda x: str(x)[0:4]+'-'+str(x)[4:6]+'-'+str(x)[6:8]+'-'+str(x)[8:])
# area_passenger_index['date/hour'] = pd.to_datetime(area_passenger_index['date/hour'])
area_passenger_index.shape

(717840, 5)

In [80]:
area_passenger_index.iloc[0:5]

,id,index,month,day,hour
0,1,1.8,1,17,0
1,1,1.5,1,17,1
2,1,1.3,1,17,2
3,1,1.3,1,17,3
4,1,1.7,1,17,4


In [81]:
### 将北京市重点区域信息和重点区域人流量合并
area_passenger_index = area_passenger_index.merge(area_passenger_info,how = 'left',on = ['id'])
area_passenger_index.shape

(717840, 12)

In [82]:
area_passenger_index.iloc[0:3]

,id,index,month,day,hour,area_name,area_type,Center_x,Center_y,Grid_x,Grid_y,area
0,1,1.8,1,17,0,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375
1,1,1.5,1,17,1,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375
2,1,1.3,1,17,2,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375


In [83]:
### 删除id, area_name, area_type
area_passenger_index = area_passenger_index.drop(['id', 'area_name', 'area_type'], axis=1)
area_passenger_index.iloc[0:10]

,index,month,day,hour,Center_x,Center_y,Grid_x,Grid_y,area
0,1.8,1,17,0,116.256586,39.986913,116.256713,39.987525,395925.609375
1,1.5,1,17,1,116.256586,39.986913,116.256713,39.987525,395925.609375
2,1.3,1,17,2,116.256586,39.986913,116.256713,39.987525,395925.609375
3,1.3,1,17,3,116.256586,39.986913,116.256713,39.987525,395925.609375
4,1.7,1,17,4,116.256586,39.986913,116.256713,39.987525,395925.609375
5,1.2,1,17,5,116.256586,39.986913,116.256713,39.987525,395925.609375
6,2.9,1,17,6,116.256586,39.986913,116.256713,39.987525,395925.609375
7,4.7,1,17,7,116.256586,39.986913,116.256713,39.987525,395925.609375
8,6.1,1,17,8,116.256586,39.986913,116.256713,39.987525,395925.609375
9,5.2,1,17,9,116.256586,39.986913,116.256713,39.987525,395925.609375


In [84]:
### 对坐标和面积做归一化
sc = MinMaxScaler()
sc.fit(area_passenger_index[['Center_x', 'Center_y', 'Grid_x', 'Grid_y', 'area']])
area_passenger_index[['Center_x', 'Center_y', 'Grid_x', 'Grid_y', 'area']] = sc.transform(area_passenger_index[['Center_x', 'Center_y', 'Grid_x', 'Grid_y', 'area']])

In [86]:
### 打乱顺序
area_passenger_index = shuffle(area_passenger_index).reset_index(drop=True)
### 提出特征和label
label = area_passenger_index['index']
features  = area_passenger_index.drop(['index'], axis=1)

In [87]:
features.iloc[0:5]

,month,day,hour,Center_x,Center_y,Grid_x,Grid_y,area
0,1,20,15,0.704571,0.278811,0.703781,0.279100,0.003549
1,2,5,22,0.508123,0.458757,0.508520,0.458611,0.000163
2,1,28,15,0.900823,0.903021,0.901164,0.902406,0.005632
3,1,31,17,0.927608,0.446886,0.927536,0.445391,0.011926
4,2,1,22,0.415220,0.964700,0.414568,0.966379,0.009625


In [92]:
### 定义模型参数
params = {'num_leaves': 38,
          'min_data_in_leaf': 50,
          'objective': 'regression',
          'max_depth': -1,
          'learning_rate': 0.02,
          "min_sum_hessian_in_leaf": 6,
          "boosting": "gbdt",
          "feature_fraction": 0.9,
          "bagging_freq": 1,
          "bagging_fraction": 0.7,
          "bagging_seed": 11,
          "lambda_l1": 0.1,
          "verbosity": -1,
          "nthread": 4,
          'metric': 'mae',
          "random_state": 2019,
#           'device': 'gpu'
          }

In [93]:
def r_mean_absolute_percentage_error(y_true, y_pred):
    return np.sqrt(np.mean((y_true-y_pred)**2))

In [97]:
folds = KFold(n_splits=5, shuffle=True, random_state=2019)

In [98]:
### K 折
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_x)):
    print("fold {}".format(fold_ + 1))
    trn_data = lgb.Dataset(features.iloc[trn_idx], label=label.iloc[trn_idx])
    val_data = lgb.Dataset(features.iloc[val_idx], label=label.iloc[val_idx])
    num_round = 1000
    clf = lgb.train(params,
                    trn_data,
                    num_round,
                    valid_sets=[trn_data, val_data],
                    verbose_eval=200,
                    early_stopping_rounds=200)
    pred = clf.predict(features.iloc[val_idx], num_iteration=clf.best_iteration)
    print('RMSE:', r_mean_absolute_percentage_error(y_true=label.iloc[val_idx], y_pred=pred))
    print('Score:', 1/(1+r_mean_absolute_percentage_error(y_true=label.iloc[val_idx], y_pred=pred)))

fold 1
Training until validation scores don't improve for 200 rounds
[200]	training's l1: 16.1376	valid_1's l1: 16.2097
[400]	training's l1: 12.8509	valid_1's l1: 12.9447
[600]	training's l1: 11.1913	valid_1's l1: 11.2909
[800]	training's l1: 10.2075	valid_1's l1: 10.3177
[1000]	training's l1: 9.38185	valid_1's l1: 9.49428
Did not meet early stopping. Best iteration is:
[1000]	training's l1: 9.38185	valid_1's l1: 9.49428
RMSE: 22.00117211298944
Score: 0.04347604526793965
fold 2
Training until validation scores don't improve for 200 rounds
[200]	training's l1: 15.9958	valid_1's l1: 16.1521
[400]	training's l1: 12.6786	valid_1's l1: 12.8073
[600]	training's l1: 11.1071	valid_1's l1: 11.2312
[800]	training's l1: 10.1123	valid_1's l1: 10.2369
[1000]	training's l1: 9.30492	valid_1's l1: 9.43417
Did not meet early stopping. Best iteration is:
[1000]	training's l1: 9.30492	valid_1's l1: 9.43417
RMSE: 21.52018355788359
Score: 0.04440461142022673
fold 3
Training until validation scores don't im

In [122]:
### submit
sub = pd.read_csv('/home/poac/code/Block_predict/submit/test_submit_example.csv', names=['id', 'date/hour', 'index'])
submit = sub.copy(deep=True)

In [123]:
sub.iloc[0:5]

,id,date/hour,index
0,1,2020021600,1.5
1,1,2020021601,1.5
2,1,2020021602,1.5
3,1,2020021603,1.5
4,1,2020021604,1.5


In [124]:
### merge
sub = sub.merge(area_passenger_info, how='left', on=['id'])
### 对时间戳进行处理
sub['month'] = sub['date/hour'].apply(lambda x: int(''.join(str(x)[4:6])))
sub['day'] = sub['date/hour'].apply(lambda x: int(''.join(str(x)[6:8])))
sub['hour'] = sub['date/hour'].apply(lambda x: int(str(x)[8:]))

### 删除原时间戳
sub = sub.drop(['date/hour'], axis=1)

### 删除id, area_name, area_type, index
sub = sub.drop(['id', 'area_name', 'area_type', 'index'], axis=1)

### 归一化
sub[['Center_x', 'Center_y', 'Grid_x', 'Grid_y', 'area']] = sc.transform(sub[['Center_x', 'Center_y', 'Grid_x', 'Grid_y', 'area']])
sub.iloc[0:5]

,id,date/hour,index,area_name,area_type,Center_x,Center_y,Grid_x,Grid_y,area
0,1,2020021600,1.5,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375
1,1,2020021601,1.5,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375
2,1,2020021602,1.5,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375
3,1,2020021603,1.5,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375
4,1,2020021604,1.5,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375


In [126]:
### 对时间戳进行处理
sub['month'] = sub['date/hour'].apply(lambda x: int(''.join(str(x)[4:6])))
sub['day'] = sub['date/hour'].apply(lambda x: int(''.join(str(x)[6:8])))
sub['hour'] = sub['date/hour'].apply(lambda x: int(str(x)[8:]))

### 删除原时间戳
sub = sub.drop(['date/hour'], axis=1)

In [127]:
sub.iloc[0:5]

,id,index,area_name,area_type,Center_x,Center_y,Grid_x,Grid_y,area,month,day,hour
0,1,1.5,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375,2,16,0
1,1,1.5,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375,2,16,1
2,1,1.5,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375,2,16,2
3,1,1.5,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375,2,16,3
4,1,1.5,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375,2,16,4


In [128]:
### 删除id, area_name, area_type, index
sub = sub.drop(['id', 'area_name', 'area_type', 'index'], axis=1)

In [129]:
### 归一化
sub[['Center_x', 'Center_y', 'Grid_x', 'Grid_y', 'area']] = sc.transform(sub[['Center_x', 'Center_y', 'Grid_x', 'Grid_y', 'area']])
sub.iloc[0:5]

,Center_x,Center_y,Grid_x,Grid_y,area,month,day,hour
0,0.349784,0.605464,0.349485,0.606595,0.017824,2,16,0
1,0.349784,0.605464,0.349485,0.606595,0.017824,2,16,1
2,0.349784,0.605464,0.349485,0.606595,0.017824,2,16,2
3,0.349784,0.605464,0.349485,0.606595,0.017824,2,16,3
4,0.349784,0.605464,0.349485,0.606595,0.017824,2,16,4


In [130]:
### 预测
pred = clf.predict(sub)

In [131]:
### 将预测结果作为index放入submit
submit['index'] = pred

In [136]:
submit.iloc[0:5]

,id,date/hour,index
0,1,2020021600,4.555173
1,1,2020021601,1604.015422
2,1,2020021602,1604.015422
3,1,2020021603,1604.015422
4,1,2020021604,1604.015422


In [137]:
### 保存
submit.to_csv('/home/poac/code/Block_predict/submit/block_predict_result.csv', index=None, header=None)